In [ ]:

데이터 불러오기부터
가. 라이브러리 Import

In [ ]:
import pandas as pd
import numpy as np

In [ ]:

# 나. 데이터 불러오기
df_feature = pd.read_csv("onenavi_train_feature.csv",sep="|")
df_target = pd.read_csv("onenavi_train_target.csv",sep="|")

In [ ]:

1. Train/Test Data Split
Overfitting을 방지하기위해 우리는 데이터 셋을 분할합니다.
KeyPoint : 학습 대상으로 정제한 데이터를 Train/Test 데이터로 분할할 수 있다.

In [ ]:
from sklearn.model_selection import train_test_split

# train_test_split : 테스트 데이터 20%
train_x, test_x, train_y, test_y = train_test_split(df_feature, df_target, test_size=0.20, random_state=42)

In [ ]:
train_x

In [ ]:

[train_test_split Option 설명]

test_size : 전체에서 test 데이터 비율을 주는 옵션입니다.(default 0.25)
shuffle : 분할 이전에 섞을지 여부를 정해주는 옵션입니다.(default True)
stratify : target으로 지정해주면 각 class 비율을 train/test에 적절하게 분배해줍니다. 즉, 쏠림 방지가 가능합니다.(default None)
random_state : 분할하는 데이터를 섞을 때 기준(?)이 되어주는 값으로 모델 최적화(튜닝)를 할때는 이 값을 고정시켜 두어야 데이터셋이 변경되지 않아 정확한 비교가 가능합니다.

In [ ]:

2. Modeling
이제 모델링을 해봅시다!
KeyPoint : 머신러닝 라이브러리를 토대로 모델링을 할 수 있다.
[공식 Document]

sklearn(https://scikit-learn.org/stable/user_guide.html)
sklearn.linear_model.LinearRegression(https://scikit-learn.org/stable/modules/generated/sklearn.linear_model.LinearRegression.html#sklearn.linear_model.LinearRegression)
sklearn.ensemble.RandomForestRegressor(https://scikit-learn.org/stable/modules/generated/sklearn.ensemble.RandomForestRegressor.html#sklearn.ensemble.RandomForestRegressor)
sklearn.ensemble.GradientBoostingRegressor(https://scikit-learn.org/stable/modules/generated/sklearn.ensemble.GradientBoostingRegressor.html#sklearn.ensemble.GradientBoostingRegressor)
xgboost(https://xgboost.readthedocs.io/en/latest/)
[여기서 짚고 넘어갈 것!]

모델의 평가 기준은 RMSE와 R-squared Score로 한다.(다른 평가 기준도 있으나, 본 실습에서는 두 가지로 진행)
RMSE(Root Mean Squared Error) : 그대로 직역하면 Error(예측 값-실제 값) 제곱의 평균에 루트한 값으로 모델의 예측 값과 실제 값의 차이를 표현
R-squared Score : 우리말로 결정계수라고 할 수 있는데, 모델의 설명력을 이야기해주는 지표이다.1에 가까울수록 모델의 설명력이 높다고 할 수 있음

### 가. LinearRegression 부터 시작해봅시다.
#### 통계와 기계학습에서 사용되는 전통적인 기법
#### 단순하게(조금은 거칠게) 표현하면  모델이 만들어낸 선형방정식과 실제 값의 오차를 기준으로 제일 작은 산식을 도출하는 방법이라고 할 수 있다.
![image.png](attachment:image.png)

In [ ]:
!pip install statsmodels

In [ ]:
# 통계기법에서 LinearRegression
import statsmodels.api as sm

results = sm.OLS(train_y, train_x).fit()

results.summary()

# *** p<0.001, ** p<0.01, * p<0.05
# https://stats.stackovernet.xyz/ko/q/37406

In [ ]:
# 기계학습에서 LinearRegression
from sklearn.linear_model import LinearRegression as lr
from sklearn.metrics import roc_auc_score, accuracy_score, mean_squared_error, r2_score

model=lr()
model.fit(train_x, train_y)

print("모델의 회귀계수는 : ", model.coef_, "이고 모델의 절편은 : ",model.intercept_)

pred_y = model.predict(test_x)
print("RMSE on Test set : {0:.5f}".format(mean_squared_error(test_y,pred_y)**0.5))
print("R-squared Score on Test set : {0:.5f}".format(r2_score(test_y,pred_y)))

In [ ]:

나. 다음은 앙상블 기법을 토대로 랜덤포레스트와 그라디언부스팅, XG부스팅을 활용해봅시다.
본격적인 활용 이전에 앙상블 기법이 무엇인지 잠깐만 살펴보고 진행하겠습니다.
앙상블은 우리가 흔히 알고 있는 음악에서 사용되는 용어의 의미와 비슷하게 여러 모델을 이용해서 조화시키는 것으로 설명할 수 있습니다.
이를 통해 여러 모델의 장점을 모아 전반적인 오류를 줄여주고 각 모델의 한계를 극복할 수 있습니다.
앙상블은 크게 Bagging과 Boosting으로 구분해서 볼 수 있습니다.
Bagging : Bootstrap Aggregating의 줄임말로 그대로 풀어보면 여러 개의 bootstrap을 생성하고 합쳐서(Aggregating) 최종 예측 모델을 산출하는 방법이다. 아래는 Bagging을 활용한 랜덤포레스트를 이해하기 쉽도록 도식화 한 것이다.


나. 다음은 앙상블 기법을 토대로 랜덤포레스트와 그라디언부스팅, XG부스팅을 활용해봅시다.
#### 본격적인 활용 이전에 앙상블 기법이 무엇인지 잠깐만 살펴보고 진행하겠습니다.
* 앙상블은 우리가 흔히 알고 있는 음악에서 사용되는 용어의 의미와 비슷하게 여러 모델을 이용해서 조화시키는 것으로 설명할 수 있습니다.
* 이를 통해 여러 모델의 장점을 모아 전반적인 오류를 줄여주고 각 모델의 한계를 극복할 수 있습니다.
* 앙상블은 크게 Bagging과 Boosting으로 구분해서 볼 수 있습니다.
* Bagging : Bootstrap Aggregating의 줄임말로 그대로 풀어보면 여러 개의 bootstrap을 생성하고 합쳐서(Aggregating) 최종 예측 모델을 산출하는 방법이다. 아래는 Bagging을 활용한 랜덤포레스트를 이해하기 쉽도록 도식화 한 것이다.
![image.png](attachment:image.png)
* Boosting : Bagging이 만든 여러 개의 bootstrap이 서로 영향을 주지 않는 병렬학습이라면 Boosting은 여러 모델을 순차적으로 학습해서 이전 모델의 결과를 바탕으로 다음 모델을 학습하는 기법이다. 이 과정을 통해 모델은 틀린 부분에 가중치를 부여하는 방식으로 오류를 더 잘 잡아낼 수 있도록 고안된 기법인데, Bagging과의 차이를 거칠게 표현하면 Bagging이 일반적인 모델을 여러개 생성하는데 집중한다면 Boosting은 정답을 찾기 어려운 문제에 집중한다고 할 수 있다. 아래는 부스팅을 이해하기 쉽게 도식화 한 것이다.
![image-2.png](attachment:image-2.png)


In [ ]:

1) 렌덤포레스트
배깅의 일종으로 의사결정나무(Decision Tree) 여러 개를 모아서 숲을 랜덤으로 구성하고 이를 종합해서 최종 모델을 산출하는 기법이라고 할 수 있다.
[주요 하이퍼 파라미터]

n_estimators : 결정트리의 갯수를 지정하는 것인데, 많을 수록 좋은 결과 값을 기대할 수 있으나, 시간도 비례해서 증가할 수 있다는 것을 명심해야한다.(default 10)
max_depth : 트리의 최대 깊이을 설정하는 것으로 많이 깊어질수록 과적합의 가능성이 높아진다.(default None)
min_samples_split : 노드를 나눌 수 있는 최소 데이터 수(default 2)
min_samples_leaf : 잎이 될 수 있는 최소 데이터 수(default 1)

In [ ]:
from sklearn.ensemble import RandomForestRegressor as rfr
from sklearn.metrics import roc_auc_score, accuracy_score, mean_squared_error, r2_score

# 다차원 배열을 1차원으로 평평하게 만들어주기!
train_y = np.ravel(train_y, order='C')

model=rfr(n_estimators=100,max_depth=5,min_samples_split=30,min_samples_leaf=15)
model.fit(train_x, train_y)

pred_y = model.predict(test_x)
print("RMSE on Test set : {0:.5f}".format(mean_squared_error(test_y,pred_y)**0.5))
print("R-squared Score on Test set : {0:.5f}".format(r2_score(test_y,pred_y)))

In [ ]:
# Feature의 중요도 확인
import matplotlib.pyplot as plt
import seaborn as sns

rf_importances_values = model.feature_importances_
rf_importances = pd.Series(rf_importances_values, index = train_x.columns)
rf_top10 = rf_importances.sort_values(ascending=False)[:10]

plt.rcParams["font.family"] = 'NanumGothicCoding'
plt.figure(figsize=(8,6))
plt.title('Top 10 Feature Importances')
sns.barplot(x=rf_top10, y=rf_top10.index,palette = "RdBu")
plt.show()

In [ ]:

2) GradientBoosting
앞선 모델의 에러를 다음 모델의 예측 값으로 활용하면서 가중치 업데이트 하는데 경사하강법(Gradient Descent)를 활용해서 최적 모델을 만드는 기법
image.png

[주요 하이퍼 파라미터]

loss : 경사하강법에서 사용할 손실함수를 지정(default ls(Least Square))
learning_rate : 말그대로 학습율이다. 기존의 예측 값에 대해 학습 결과를 얼마나 업데이트(보정)할지 지정(default 0.1)
n_estimators : 부스팅 단계의 갯수를 지정하는 것인데, 많을 수록 좋은 결과 값을 기대할 수 있으나, 시간도 비례해서 증가할 수 있다는 것을 명심해야한다.(default 10)
max_depth : 트리의 최대 깊이을 설정하는 것으로 많이 깊어질수록 과적합의 가능성이 높아진다.(default None)
min_samples_split : 노드를 나눌 수 있는 최소 데이터 수(default 2)
min_samples_leaf : 잎이 될 수 있는 최소 데이터 수(default 1)

In [ ]:
from sklearn.ensemble import GradientBoostingRegressor as grb
from sklearn.metrics import roc_auc_score, accuracy_score, mean_squared_error, r2_score

# 다차원 배열을 1차원으로 평평하게 만들어주기!
train_y = np.ravel(train_y, order='C')

model=grb(n_estimators=100,learning_rate=0.1,max_depth=5,min_samples_split=30,min_samples_leaf=15)
model.fit(train_x, train_y)

pred_y = model.predict(test_x)
print("RMSE on Test set : {0:.5f}".format(mean_squared_error(test_y,pred_y)**0.5))
print("R-squared Score on Test set : {0:.5f}".format(r2_score(test_y,pred_y)))

In [ ]:
# Feature의 중요도 확인
import matplotlib.pyplot as plt
import seaborn as sns

grb_importances_values = model.feature_importances_
grb_importances = pd.Series(grb_importances_values, index = train_x.columns)
grb_top10 = grb_importances.sort_values(ascending=False)[:10]

plt.rcParams["font.family"] = 'NanumGothicCoding'
plt.figure(figsize=(8,6))
plt.title('Top 10 Feature Importances')
sns.barplot(x=grb_top10, y=grb_top10.index,palette = "RdBu")
plt.show()

In [ ]:

3) XGBoost
부스팅 모델이 매우 강력한 것은 사실이지만 단점도 존재(느리고, 과적합의 이슈), 그래서 GradientBoosting보다 빠르고 규제를 설정해서 과적합 방지가 가능한 XGBoost가 등장
[주요 하이퍼 파라미터]

booster : 사용할 부스터(default gbtree)
gamma : 가지를 칠 때 필요한 최소손실 감소로 클수록 보수적(가지를 치지 않는다)이라고 할 수 있다.(default 0)
eta : 말그대로 학습율이다. 기존의 예측 값에 대해 학습 결과를 얼마나 업데이트(보정)할지 지정(default 0.3)
n_estimators : 부스팅 단계의 갯수를 지정하는 것인데, 많을 수록 좋은 결과 값을 기대할 수 있으나, 시간도 비례해서 증가할 수 있다는 것을 명심해야한다.(default 100)
max_depth : 트리의 최대 깊이을 설정하는 것으로 많이 깊어질수록 과적합의 가능성이 높아진다.(default 6)
reg_lambda : 가중치에 대한 L2 정규화(default 1), 커질수록 오버피팅 방지, 너무 큰 가중치를 줄여줄 수 있음
reg_alpha : 가중치에 대한 L1 정규화(default 0), 커질수록 오버피팅 방지, 불필요한 가중치를 0으로 만들어서 무시
[참고] https://developers.google.com/machine-learning/crash-course/regularization-for-sparsity/l1-regularization?hl=ko
early_stopping_rounds : 조기 종료 옵션, 지정 횟수 동안 결과가 개선 되지 않으면 조기 종료 한다.

In [ ]:
from xgboost import XGBRegressor as xgb
from sklearn.metrics import roc_auc_score, accuracy_score, mean_squared_error, r2_score

# 다차원 배열을 1차원으로 평평하게 만들어주기!
train_y = np.ravel(train_y, order='C')

model=xgb(n_estimators=100,gamma=1,eta=0.1,max_depth=5,reg_lambda=5,reg_alpha=5)
model.fit(train_x, train_y)

pred_y = model.predict(test_x)
print("RMSE on Test set : {0:.5f}".format(mean_squared_error(test_y,pred_y)**0.5))
print("R-squared Score on Test set : {0:.5f}".format(r2_score(test_y,pred_y)))

In [ ]:
# Feature의 중요도 확인
import matplotlib.pyplot as plt
import seaborn as sns

xgb_importances_values = model.feature_importances_
xgb_importances = pd.Series(xgb_importances_values, index = train_x.columns)
xgb_top10 = xgb_importances.sort_values(ascending=False)[:10]

plt.rcParams["font.family"] = 'NanumGothicCoding'
plt.figure(figsize=(8,6))
plt.title('Top 10 Feature Importances')
sns.barplot(x=xgb_top10, y=xgb_top10.index,palette = "RdBu")
plt.show()

In [ ]:

다. 그럼 여러 개의 모델을 한 번에 확인해볼까요?

In [ ]:
from sklearn.linear_model import LinearRegression as lr
from sklearn.ensemble import RandomForestRegressor as rfr
from sklearn.ensemble import GradientBoostingRegressor as grb
from xgboost import XGBRegressor as xgb
from sklearn.metrics import roc_auc_score, accuracy_score, mean_squared_error, r2_score

import pickle
import joblib
import time

model_list=[
            lr(),
            rfr(),
            grb(),
            xgb()
            ]

# 다차원 배열을 1차원으로 평평하게 만들어주기!
train_y = np.ravel(train_y, order='C')

model_rslt = []
for i in range(len(model_list)):
    start_time = time.process_time()
    model = model_list[i]
    model.fit(train_x, train_y)
    end_time = time.process_time()
    joblib.dump(model, '{}_model.pkl'.format(i)) # 모델 저장, sklearn을 통해서 만들어진 모델은 pkl 파일로 저장
    print(f"* {model} 결과 시작")
    print('----  {0:.5f}sec, training complete  ----'.format(end_time-start_time))
    pred_y = model.predict(test_x)
    model_rslt.append(model)
    print("RMSE on Test set : {0:.5f}".format(mean_squared_error(test_y,pred_y)**0.5))
    print("R-squared Score on Test set : {0:.5f}".format(r2_score(test_y,pred_y)))
    print("---------------------------------------------------------------------------")